# GEORREFERENCIAMENTO OVP-DH
## Jupyter Notebook compartilhado

In [19]:
import numpy as np
import pandas as pd
import folium
from folium import plugins
from folium.plugins import MarkerCluster

### Abrindo dataset

In [20]:
data = pd.read_csv('../data/neo_geo_bo2016.csv').drop(columns=['Unnamed: 0'])

data.head()

,NUM_BO,DELEGACIA,RUBRICA,ANO,MES,CIDADE,LOGRADOURO,NUMERO_LOGRADOURO,LATITUDE,LONGITUDE,ENDERECO,CRIME,ARTIGO,DATA
0,4705,98º D.P. JARDIM MIRIAM,Roubo (art. 157),2016,6,S.PAULO,AVENIDA YERVANT KISSAJIKIAN,3837,-23.684968,-46.647860,"S.Paulo, Avenida Yervant Kissajikian",Roubo,Art. 157,2016-06-01
1,6168,33º D.P. PIRITUBA,Roubo (art. 157),2016,10,S.PAULO,RUA DONA HERMINDA BORBA ATTUY,260,-23.504276,-46.737431,"S.Paulo, Rua Dona Herminda Borba Attuy",Roubo,Art. 157,2016-10-01
2,871489,DELEGACIA ELETRONICA,Roubo (art. 157),2016,7,S.PAULO,RUA PROFESSOR BATISTA DE ANDRADE,0,-23.546011,-46.621758,"S.Paulo, Rua Professor Batista De Andrade",Roubo,Art. 157,2016-07-01
3,964,09º D.P. OSASCO,Furto (art. 155),2016,6,OSASCO,RUA MANUEL ANTÔNIO PORTELA,65,-23.529230,-46.760071,"Osasco, Rua Manuel Antônio Portela",Furto,Art. 155,2016-06-01
4,907154,DELEGACIA ELETRONICA,Furto (art. 155),2016,7,S.PAULO,RUA GUAICURUS,324,-23.523082,-46.687876,"S.Paulo, Rua Guaicurus",Furto,Art. 155,2016-07-01


## Construindo o mapa

### Definindo 'RUBRICA' como objetivo de pesquisa

In [21]:
# Pegando a média da latitudes e longitudes das ocorrências
latmean = data['LATITUDE'].mean()
lonmean = data['LONGITUDE'].mean()

# Criando o mapa
mapa_cluster = folium.Map(location=[latmean,lonmean], 
                         zoom_start=8, 
                         tiles=None, 
                         control_scale=True)

# Determinando as texturas do mapa
texturas = [('openstreetmap', 'textura: tradicional'),
            ('stamenterrain', 'textura: terreno'),
            ('stamenwatercolor','textura: desenho')]

for tile, name in texturas:
    folium.TileLayer(tiles=tile, name=name).add_to(mapa_cluster)
    
# Minimapa
minimap = plugins.MiniMap(toggle_display=True)
mapa_cluster.add_child(minimap)

# Plugins - fullscreen
plugins.Fullscreen(position='topleft').add_to(mapa_cluster)

### Definindo as cores

In [22]:
### Definindo as cores
dic_nl = {'Furto qualificado':'lightgreen',
          'Furto':'lightgreen',
          'Roubo':'darkgreen',
          'A.I.-Roubo':'darkgreen',
          'Drogas sem autorização ou em desacordo':'purple',
          'A.I.-Drogas sem autorização ou em desacordo':'purple',
          'Lesão corporal':'black',
          'A.I.-Lesão corporal':'black',
          'Lesão corporal culposa na direção de veículo automotor':'cadetblue',
          'Homicídio culposo na direção de veículo automotor':'darkblue',
          'A.I.-Estupro de vulneravel':'darkred',
          'Estupro de vulneravel':'darkred',
         }

### Plotando sub-grupos em camadas

In [23]:
### Plotando sub-grupos em camadas
all_subgroups = folium.FeatureGroup(name='Tudo')

mapa_cluster.add_child(all_subgroups);

In [24]:
#adicionando clusterizacao
mc = MarkerCluster(control=False)
mapa_cluster.add_child(mc)

#definindo a funcao para cruacao dos marcadores
def grupo_mapa(CRIME):
    group = plugins.FeatureGroupSubGroup(mc, CRIME)
    mapa_cluster.add_child(group)
    
    for i in data[data['CRIME'] == CRIME].itertuples():
        folium.Marker(location=[i.LATITUDE, i.LONGITUDE], 
                      popup=CRIME, 
                      icon=folium.Icon(color=dic_nl[CRIME], 
                                       icon='bolt',
                                       prefix='fa')).add_to(group)
    

In [25]:
#iterando a funcao sobre o set
for i in dic_nl.keys():
    grupo_mapa(i)

In [26]:
# adicionando menu
folium.LayerControl(collapsed=True).add_to(mapa_cluster);

### Visualizando o mapa

In [27]:
mapa_cluster

### Salvando o mapa

In [120]:
mapa_cluster.save('../examples/mapa_cluster.html')